In [45]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import ElementNotVisibleException, ElementNotSelectableException, NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.service import Service as FirefoxService
from unidecode import unidecode
from pathlib import Path
from datetime import datetime, timedelta
from lingua import Language, LanguageDetectorBuilder
import pandas as pd
import json
import time

In [46]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('disable-translate')
chrome_options.add_argument('--lang=fr')
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--disable-blink-features=AutomationControlled')
chrome_options.add_extension("./canvas_blocker.crx")

In [47]:
url = "https://www.tripadvisor.fr/Hotel_Review-g488104-d316747-Reviews-LUX_Grand_Gaube-Grand_Gaube.html"

In [48]:
driver = webdriver.Chrome(options=chrome_options)

In [49]:
driver.get(url)
driver.get(url)

In [ ]:
element = driver.find_element(By.XPATH, "//div[@data-test-target='reviews-tab']")
driver.execute_script('arguments[0].scrollIntoView({ behavior: "smooth", block: "center", inline: "center" })', element)

In [ ]:
driver.find_elements(By.XPATH, "//div[@data-test-target='HR_CC_CARD']")

In [ ]:
div.IaVba.F1 title

	a.blnum

In [ ]:
{"_id":"tripadvisor","startUrl":["https://www.tripadvisor.fr/Hotel_Review-g488104-d316747-Reviews-LUX_Grand_Gaube-Grand_Gaube.html"],"selectors":[{"id":"reviews","parentSelectors":["_root"],"type":"SelectorElement","selector":"div.azLzJ","multiple":true},{"id":"author","parentSelectors":["reviews"],"type":"SelectorText","selector":"a.blnum","multiple":false,"regex":""},{"id":"rating","parentSelectors":["reviews"],"type":"SelectorText","selector":"div.IaVba.F1 title","multiple":false,"regex":""},{"id":"comment","parentSelectors":["reviews"],"type":"SelectorText","selector":".orRIx span","multiple":false,"regex":""},{"id":"date","parentSelectors":["reviews"],"type":"SelectorText","selector":"span.iSNGb","multiple":false,"regex":""},{"id":"scroll","parentSelectors":["_root"],"type":"SelectorElementScroll","selector":"div.mXiGB","multiple":false,"delay":5000,"elementLimit":500},{"id":"next","parentSelectors":["_root","next"],"paginationType":"linkFromHref","type":"SelectorPagination","selector":"a.unMkR"}]}